In [1]:
import torch
import math
from d2l import torch as d2l
from torch import nn
import pandas as pd
from MyKu import processing
from MyKu import training

In [2]:
glove_embedding = processing.TokenEmbedding('glove.42b.300d')

In [3]:
class BiRNN(nn.Module):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers, **kwargs):
        super(BiRNN, self).__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.encoder = nn.LSTM(
            embed_size, num_hiddens, num_layers=num_layers, bidirectional=True, dropout=0.5)
        self.decoder = nn.Linear(num_hiddens * 4, 2)

    def forward(self, inputs):
        embeddings = self.embedding(inputs.T)
        self.encoder.flatten_parameters()
        outputs, _ = self.encoder(embeddings)
        encoding = torch.cat((outputs[0], outputs[-1]), dim=1)
        outs = self.decoder(encoding)
        return outs


In [4]:
batch_size = 32
train_iter, test_iter, vocab = processing.load_exist2021_data(batch_size)
embed_size, num_hiddens, num_layers = 300, 200, 5

devices = d2l.try_all_gpus()
net = BiRNN(len(vocab), embed_size, num_hiddens, num_layers)
print(net.eval())
def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.xavier_normal_(m.weight)
    if type(m) == nn.LSTM:
        for param in m._flat_weights_names:
            if "weight" in param:
                nn.init.xavier_normal_(m._parameters[param])

net.apply(init_weights)
embeds = glove_embedding[vocab.idx_to_token]
embeds.shape
net.embedding.weight.data.copy_(embeds)
net.embedding.weight.requires_grad = False

lr, num_epochs = 0.0005, 20
trainer = torch.optim.Adam(net.parameters(), lr=lr)
loss = nn.CrossEntropyLoss(reduction="none")
training.train(net, train_iter, test_iter, loss, trainer, num_epochs, devices)


ValueError: too many values to unpack (expected 2)